# Assessment Task 1 Part B
## Computer Vision
Juan Pablo Herrera Alvarez

In [1]:
# Imports General Packages 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Import System packages 
import os

# Import Keras functions
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout,Conv2D, Activation, Flatten, MaxPool2D, BatchNormalization, GlobalAveragePooling2D
from keras.optimizers import Adam

# Import Keras Applications
from keras.applications.resnet50 import ResNet50

In [2]:
# Loads Data and Checks the Shape
'''In this part is important to mention that I have previouly downloaded
the mnist repository in my working directory'''
from fashion_mnist.utils import mnist_reader
X_train, y_train = mnist_reader.load_mnist('fashion_mnist/data/fashion', kind='train')
X_test, y_test = mnist_reader.load_mnist('fashion_mnist/data/fashion', kind='t10k')

## Checkds the shape of the imported data
'''After taking a look at the shape dof the data it is evident that a 
reshape is needed'''
#print(X_train.shape)
#print(X_test.shape)

y_train = y_train.reshape(y_train.shape[0], 1)
y_test = y_test.reshape(y_test.shape[0], 1)

print(y_train.shape)

(60000, 1)


In [3]:
# Data wrangling
'''The goal here is to transform the data to the shape (N,28,28)'''
## first the take a look at how the data should look
#exp = X_train[108, :].reshape(28,28) 
#exp.shape
#plt.imshow(exp, cmap='PuBu')
#plt.axis('off')
#plt.show() # It's a shoe!!

## Reshapes the data sets
shape = (28, 28) # Target shape
X_train = X_train.reshape(X_train.shape[0], *shape) 
X_test = X_test.reshape(X_test.shape[0],*shape) 

#print(X_train.shape)
#print(X_test.shape)

## Check that the reshaped dataset produce the same images
#exp2 = X_train[108,:,:,0]
#plt.imshow(exp2, cmap='PuBu')
#plt.axis('off')
#plt.show() # Success, they match

In [4]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(60000, 28, 28)
(10000, 28, 28)
(60000, 1)
(10000, 1)


## Part 1 - Transfer Learning

In [5]:
# Preporcesing for ResNet50 
'''ResNet50 requires a minimun of three input channels and the image size
cannot be smaller than 32 by 32'''
## Stacking to increase the number of channels 
X_train = np.stack((X_train, )*3, axis = -1)
X_test = np.stack((X_test, )*3, axis = -1)
#print(X_train.shape) 

## Padding to increase the image size from 28 to 32
X_train50 = np.pad(X_train, ((0,0),(2,2),(2,2),(0,0)), mode='constant',constant_values= 0)
X_test50 = np.pad(X_test, ((0,0),(2,2),(2,2),(0,0)), mode='constant',constant_values= 0)

#print(X_train50.shape)


In [6]:
print(X_train50.shape)
print(X_test50.shape)
print(y_train.shape)
print(y_test.shape)

(60000, 32, 32, 3)
(10000, 32, 32, 3)
(60000, 1)
(10000, 1)


### Model 1: ResNet50 with Imagenet Weights 

In [7]:
# Base ResNet50 model
model1 = ResNet50(weights='imagenet', include_top=False, 
                  input_shape=[32, 32, 3])

base = model1.output
#base = GlobalAveragePooling2D()(base)
base = Flatten()(base)
prediction = Dense(10, activation='softmax')(base) # 10 different classes

## Freeze all layers
for layer in model1.layers:
    layer.trainable = False

model = Model(model1.input, prediction)


model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam', metrics=['accuracy'])
#model.summary()

Instructions for updating:
Colocations handled automatically by placer.


c:\users\mutaz\appdata\local\programs\python\python36\lib\site-packages\keras_applications\resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


94658560/94653016 [==============================] - 135s 1us/step


In [8]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 38, 38, 3)    0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 16, 16, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 16, 16, 64)   256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [ ]:
model.fit(X_train50, y_train, batch_size = 32, 
          epochs = 20, verbose = 1, 
          validation_data = (X_test50, y_test))

Instructions for updating:
Use tf.cast instead.
Train on 60000 samples, validate on 10000 samples
Epoch 1/20
 4096/60000 [=>............................] - ETA: 21:44 - loss: 1.5890 - acc: 0.5110

### Model 2: ResNet50 with 2 dense extra layers

### Model 3: Re-trained ResNet50 with the two extra layers

## Part 2 - Custom Architecture

## Part 3 - Improvements